
# 奇异值分解（SVD）：它到底是什么？怎么用？有什么价值？

奇异值分解（Singular Value Decomposition），简称 **SVD**，是机器学习、信号处理、图像压缩、推荐系统中最重要的数学工具之一。

一句话解释：

> **SVD = 把一个复杂矩阵拆成：方向 × 强度 × 方向，找出数据里最重要的结构。**


---

# 1. SVD 到底在做什么？

假设你有一个矩阵 (A)，它可以是：

* 一张图片（像素矩阵）
* 一个用户-商品打分表
* 一个文本的词频矩阵
* 一个机器学习数据集（样本 × 特征）

SVD 会把它拆成三个矩阵：

[
A = U \Sigma V^T
]

这三个矩阵分别是什么？一句话概括：

| 矩阵       | 意义（通俗）        | 数学意义              |
| -------- | ------------- | ----------------- |
| (U)      | 输出方向（行空间的主方向） | 右侧矩阵 (AA^T) 的特征向量 |
| (\Sigma) | 重要性（每个方向的权重）  | 奇异值，非负、降序，能量大小    |
| (V)      | 输入方向（列空间的主方向） | 右侧矩阵 (A^TA) 的特征向量 |

如果把 SVD 拆成一句话：

> **V 告诉你数据“主要从哪些方向变化”，
> U 告诉你这些变化如何映射到输出空间，
> Σ 告诉你哪些变化更重要。**

---

# 2. 为什么 SVD 能“看懂数据结构”？

在现实数据中，很多东西都是“有主次之分”的：

* 图像里重要的是轮廓，不重要的是噪点
* 推荐系统里有些用户行为很典型，有些不重要
* 文本中有些词表达主题，有些词只是停用词
* 数据集中有些特征强关联，有些只是噪声

SVD 的奇异值（Σ 的对角线）从大到小排列：

> 大奇异值 → 数据的主要结构
> 小奇异值 → 细节或噪声

因此，只保留前 (k) 个奇异值：

$$
A \approx U_k \Sigma_k V_k^T
$$`

就是在用最重要的信息近似原数据，实现：

* 降维
* 压缩
* 去噪
* 提取主结构

这也是 PCA 的数学基础。

---

# 3. SVD 的核心价值（为什么那么重要？）

### 1）它是最“通用”的分解

* 特征值分解只能用于方阵
* SVD 适用于任意矩阵（m×n）

是工业界最稳、最强、适用最广的数学工具。

---

### 2）它天然具备降维能力

奇异值按信息量从大到小排列，截断就是降维。

例如保留前 1 个奇异值就能还原一张图片的轮廓。

---

### 3）它是 PCA 的直接公式

从 (A) 做 SVD：

$$
A = U \Sigma V^T
$$

PCA 用的主成分就是 (V) 的前几列。

---

### 4）它让“矩阵理解世界”成为可能

任何数据，只要能表示成矩阵——就能用 SVD 理解它。

例如：

* 用户 × 商品
* 文档 × 词
* 图片 × 像素
* 样本 × 特征


# 实际案例

**输出解释**：
- 原始矩阵 $A$ 被分解为 $U$、$\Sigma$（由 `sigma` 表示）和 $V^T$。
- 通过 $U \cdot \Sigma \cdot V^T$ 的矩阵乘法，可以完全重构原始矩阵 $A$，几乎没有信息损失（可能有非常小的数值误差）。

## 矩阵降维案例


In [1]:
import numpy as np

# 创建一个矩阵 A (5x3)
A = np.array([[1, 2, 3],
              [4, 5, 6],
              [7, 8, 9],
              [10, 11, 12],
              [13, 14, 15]])
print("原始矩阵 A:")
print(A)

# 进行 SVD 分解
U, sigma, Vt = np.linalg.svd(A, full_matrices=False)
print("\n奇异值 sigma:")
print(sigma)

# 保留前 k=1 个奇异值进行降维
k = 1
U_k = U[:, :k]  # 取 U 的前 k 列，因为要保持行数不变
sigma_k = sigma[:k]  # 取前 k 个奇异值
Vt_k = Vt[:k, :]  # 取 Vt 的前 k 行，因为要保持列数不变

# 近似重构矩阵 A,常用于信号or图像筛除噪声
A_approx = U_k @ np.diag(sigma_k) @ Vt_k
print("\n保留前", k, "个奇异值后的近似矩阵 A_approx:")
print(A_approx)

# 计算近似误差
error = np.linalg.norm(A - A_approx, 'fro') / np.linalg.norm(A, 'fro')
print("\n近似误差 (Frobenius 范数相对误差):", error)


原始矩阵 A:
[[ 1  2  3]
 [ 4  5  6]
 [ 7  8  9]
 [10 11 12]
 [13 14 15]]

奇异值 sigma:
[3.51826483e+01 1.47690770e+00 1.65564886e-15]

保留前 1 个奇异值后的近似矩阵 A_approx:
[[ 1.85152908  2.05208851  2.25264793]
 [ 4.5411984   5.03310541  5.52501242]
 [ 7.23086771  8.01412231  8.7973769 ]
 [ 9.92053702 10.99513921 12.06974139]
 [12.61020633 13.9761561  15.34210588]]

近似误差 (Frobenius 范数相对误差): 0.04194136031471533




---
在机器学习中，如果对训练集进行 SVD 降维后训练模型，而测试集的特征数量与降维后的训练集不一致（测试集仍保持原始特征数量），该如何处理？
### 1. 问题分析
- **训练集降维**：假设训练集有 1000 个样本，50 个特征，通过 SVD 降维后保留 $k=10$ 个特征，得到形状为 `(1000, 10)` 的新数据。模型基于这 10 个特征进行训练。
- **测试集问题**：测试集假设有 200 个样本，仍然是 50 个特征。如果直接输入测试集到模型中，特征数量不匹配（模型期望 10 个特征，测试集有 50 个），会导致错误。
- **核心问题**：如何确保测试集也能被正确地降维到与训练集相同的 $k$ 个特征空间？

---

### 2. 解决方案：对测试集应用相同的变换
在机器学习中，降维（如 SVD、PCA 等）是一种数据预处理步骤。训练集和测试集必须经过**相同的变换**，以确保数据分布一致。具体到 SVD，步骤如下：
1. **训练阶段**：对训练集 $X_{train}$ 进行 SVD 分解，得到 $U$, $\Sigma$, 和 $V^T$，并保存 $V^T$ 矩阵（或其前 $k$ 行）用于降维变换。
2. **测试阶段**：使用从训练集得到的 $V^T$ 矩阵，将测试集 $X_{test}$ 投影到相同的低维空间，得到降维后的测试数据。
3. **原因**：$V^T$ 矩阵定义了从原始特征空间到低维特征空间的映射关系，测试集必须使用相同的映射以保持一致性。

数学上，假设训练集 SVD 分解为 $X_{train} = U \Sigma V^T$，我们保留前 $k$ 个奇异值对应的 $V_k^T$（形状为 $k \times 50$）。测试集降维公式为：
$$X_{test\_reduced} = X_{test} \cdot V_k^T.T$$
其中 $V_k^T.T$ 是 $V_k^T$ 的转置（形状为 $50 \times k$），$X_{test}$ 是形状为 `(n_test, 50)` 的测试集矩阵，降维后 $X_{test\_reduced}$ 的形状为 `(n_test, k)`。

---

### 3. 为什么不能对测试集单独做 SVD？
- 如果对测试集单独进行 SVD，会得到不同的 $V^T$ 矩阵，导致测试集和训练集的低维空间不一致，模型无法正确处理测试数据。
- 训练集的 $V^T$ 矩阵代表了训练数据的特征映射规则，测试集必须遵循相同的规则，否则会引入数据泄漏或不一致性问题。

---

### 4. 代码示例：训练集和测试集的 SVD 降维
以下是一个完整的代码示例，展示如何对训练集进行 SVD 降维，训练模型，并对测试集应用相同的降维变换。


---
Day_24 实战部分我们沿用 day20 中"先对训练集做 SVD 再对测试集应用相同投影"的策略, 并选择 Kaggle 心脏病数据集 `heart.csv` 作为真实案例:
1. 标准化后先训练逻辑回归得到基准准确率。
2. 对训练集执行 SVD, 依据累计能量阈值选择前 `k` 个奇异值, 构造投影矩阵。
3. 用相同投影处理测试集, 再次训练/评估, 对比精度变化。

下面的代码块给出了完整流程。


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# 1. 读取 heart.csv 并拆分特征与标签
df = pd.read_csv('heart.csv')
X = df.drop(columns=['target'])
y = df['target']

# 2. 训练集/测试集划分 + 标准化（保证 SVD 与逻辑回归稳定）
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y,
)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
print(f"训练集形状: {X_train.shape}, 测试集形状: {X_test.shape}")

# 3. 直接训练逻辑回归, 得到基准准确率
baseline_model = LogisticRegression(max_iter=1000, random_state=42)
baseline_model.fit(X_train_scaled, y_train)
baseline_acc = accuracy_score(y_test, baseline_model.predict(X_test_scaled))
print(f"基准模型（未降维）准确率: {baseline_acc:.4f}")

# 4. 对训练集做 SVD, 并找到覆盖 90% 能量所需的奇异值数量
U_train, sigma_train, Vt_train = np.linalg.svd(X_train_scaled, full_matrices=False)
energy = sigma_train ** 2
energy_ratio = energy / energy.sum()
cumulative_energy = np.cumsum(energy_ratio)
energy_threshold = 0.9
k = int(np.searchsorted(cumulative_energy, energy_threshold) + 1)
print(f"使用前 {k} 个奇异值即可覆盖 {energy_threshold*100:.0f}% 的能量")

# 5. 使用相同的投影矩阵对训练集和测试集降维
Vt_k = Vt_train[:k, :]
X_train_reduced = X_train_scaled @ Vt_k.T
X_test_reduced = X_test_scaled @ Vt_k.T

svd_model = LogisticRegression(max_iter=1000, random_state=42)
svd_model.fit(X_train_reduced, y_train)
svd_acc = accuracy_score(y_test, svd_model.predict(X_test_reduced))
print(f"SVD 降维后准确率: {svd_acc:.4f}")
print(f"准确率变化: {svd_acc - baseline_acc:+.4f}")

# 6. 观察前几个主成分的累计能量, 方便选择 k
energy_report = pd.DataFrame(
    {
        'component': np.arange(1, k + 1),
        'singular_value': sigma_train[:k],
        'cumulative_energy': cumulative_energy[:k],
    }
)
print(energy_report)



训练集形状: (242, 13), 测试集形状: (61, 13)
基准模型（未降维）准确率: 0.8033
使用前 10 个奇异值即可覆盖 90% 的能量
SVD 降维后准确率: 0.7869
准确率变化: -0.0164
   component  singular_value  cumulative_energy
0          1       26.448257           0.222349
1          2       19.453446           0.342640
2          3       17.582638           0.440908
3          4       16.597867           0.528476
4          5       15.321514           0.603094
5          6       15.001419           0.674627
6          7       14.495351           0.741415
7          8       13.614129           0.800329
8          9       12.986804           0.853939
9         10       12.447085           0.903186


## heart.csv 实验结论

- 心脏病数据集中共有 13 个特征, 降维前直接训练逻辑回归的测试准确率约为 **0.8033**。
- 依据奇异值累计能量（90% 阈值）选择前 **10** 个主方向, SVD 降维后准确率约为 **0.7869**。
- 维度本身不高, 信息压缩后丢失了一些判别特征, 因此精度略微下降（约 -0.0164）。
- 当特征数远大于样本数或存在强相关性时, SVD/主成分降维通常能缓解过拟合并提升泛化, 也可用此流程快速评估最佳的 k。
